In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
#import statsmodels.api as sm
from math import ceil
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectPercentile as SP
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from tqdm import tqdm
from scipy import stats
from scipy.special import boxcox, inv_boxcox
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
import math
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [2]:
train_df = pd.read_csv('X_train_final.csv')
test_df = pd.read_csv('X_test_final.csv')

In [3]:
y_train_lambda = 0.21963574771308522
target = train_df['price_doc']
target_boxcox = train_df['price_doc_boxcox']
train_df = train_df.drop(['price_doc', 'price_doc_boxcox'],axis = 1)
#train_df = train_df.join(target)
train_df = train_df.join(target_boxcox)

In [4]:
train_df = train_df.drop('id',axis = 1)
test_df = test_df.drop('id',axis = 1)

In [5]:
'''def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df'''

"def reduce_mem_usage(df, verbose=True):\n    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']\n    start_mem = df.memory_usage().sum() / 1024**2    \n    for col in df.columns:\n        col_type = df[col].dtypes\n        if col_type in numerics:\n            c_min = df[col].min()\n            c_max = df[col].max()\n            if str(col_type)[:3] == 'int':\n                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:\n                    df[col] = df[col].astype(np.int8)\n                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:\n                    df[col] = df[col].astype(np.int16)\n                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:\n                    df[col] = df[col].astype(np.int32)\n                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:\n                    df[col] = df[col].astype(np.int64)  \n            else:\n            

In [6]:
'''train= reduce_mem_usage(train_df)
test= reduce_mem_usage(test_df)
print("Shape of train set: ",train.shape)
print("Shape of test set: ",test.shape)'''

'train= reduce_mem_usage(train_df)\ntest= reduce_mem_usage(test_df)\nprint("Shape of train set: ",train.shape)\nprint("Shape of test set: ",test.shape)'

In [7]:
from scipy import stats
import time
import sys
import os

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, mean_squared_error

import xgboost as xgb
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR

import optuna

In [8]:
X = train_df.iloc[:,:-1]
y = train_df.iloc[:,-1]
num_trials = 20

In [9]:
def lgbm_objective(trial,data=X,target=y):
    
    X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.30, random_state=42)
    param = {
        'metric': 'rmse', 
        'random_state': 42,
        'n_estimators': 20000,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.005,0.008,0.01,0.03,0.05]),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    model = LGBMRegressor(**param)
    
    
    
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],verbose=False)
    
    preds = model.predict(X_test)
    
    rmse = mean_squared_error(y_test, preds,squared=False)
    
    return rmse

In [10]:
lgbm_study = optuna.create_study(direction='minimize')
lgbm_study.optimize(lgbm_objective, n_trials=num_trials)

[I 2022-11-21 23:53:06,749] A new study created in memory with name: no-name-e20e601d-b428-451d-8076-35424299f7fe
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\2738468821.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\2738468821.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
C:\Users\bhara\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evalu

[I 2022-11-22 00:00:28,192] Trial 5 finished with value: 13.885032995069773 and parameters: {'reg_alpha': 0.002389427421042918, 'reg_lambda': 0.013864497927558303, 'colsample_bytree': 0.9, 'subsample': 0.4, 'learning_rate': 0.05, 'max_depth': 9, 'num_leaves': 622, 'min_child_samples': 138, 'min_data_per_groups': 10}. Best is trial 3 with value: 12.874837014299866.
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\2738468821.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\2738468821.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instea

[I 2022-11-22 00:12:11,660] Trial 11 finished with value: 12.973000022543877 and parameters: {'reg_alpha': 0.1680600074336294, 'reg_lambda': 0.18079098106068195, 'colsample_bytree': 0.6, 'subsample': 1.0, 'learning_rate': 0.005, 'max_depth': 5, 'num_leaves': 741, 'min_child_samples': 81, 'min_data_per_groups': 33}. Best is trial 3 with value: 12.874837014299866.
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\2738468821.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\2738468821.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

[I 2022-11-22 00:16:16,368] Trial 17 finished with value: 12.844469658105648 and parameters: {'reg_alpha': 0.9250102888963972, 'reg_lambda': 0.0012070644986337915, 'colsample_bytree': 0.4, 'subsample': 0.5, 'learning_rate': 0.005, 'max_depth': 5, 'num_leaves': 822, 'min_child_samples': 294, 'min_data_per_groups': 82}. Best is trial 17 with value: 12.844469658105648.
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\2738468821.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\2738468821.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` inst

In [11]:
lgbm_params=lgbm_study.best_params 
lgbm_params['random_state'] = 42
lgbm_params['metric'] = 'rmse'
lgbm_params['n_estimators'] = 20000
lgbm_params


{'reg_alpha': 0.9250102888963972,
 'reg_lambda': 0.0012070644986337915,
 'colsample_bytree': 0.4,
 'subsample': 0.5,
 'learning_rate': 0.005,
 'max_depth': 5,
 'num_leaves': 822,
 'min_child_samples': 294,
 'min_data_per_groups': 82,
 'random_state': 42,
 'metric': 'rmse',
 'n_estimators': 20000}

In [12]:
def xgb_objective(trial,data=X,target=y):
    
    X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.30, random_state=42)
    param = {
            'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
            'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
            'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
            'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
            'learning_rate': trial.suggest_categorical('learning_rate', [0.005,0.008,0.01,0.03,0.05]),
            'n_estimators': 10000,
            'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13]),
            'random_state': 42,
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 300)
        }
    model = XGBRegressor(**param)  
    
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(X_test)
    
    rmse = mean_squared_error(y_test, preds,squared=False)
    
    return rmse

In [13]:
xgb_study = optuna.create_study(direction='minimize')
xgb_study.optimize(xgb_objective, n_trials=num_trials)

[I 2022-11-22 00:18:32,315] A new study created in memory with name: no-name-f4230263-de4c-4905-9ecc-9d166c5ec20c
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\2576784288.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\2576784288.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
C:\Users\bhara\anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_

[I 2022-11-22 00:24:33,633] Trial 6 finished with value: 12.714415663613222 and parameters: {'lambda': 2.4827990514474725, 'alpha': 0.31216401684148964, 'colsample_bytree': 0.5, 'subsample': 1.0, 'learning_rate': 0.005, 'max_depth': 7, 'min_child_weight': 132}. Best is trial 6 with value: 12.714415663613222.
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\2576784288.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\2576784288.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
C:

[I 2022-11-22 00:31:26,292] Trial 13 finished with value: 12.7748771842671 and parameters: {'lambda': 0.0010267524397943792, 'alpha': 0.01866564396023856, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.01, 'max_depth': 7, 'min_child_weight': 202}. Best is trial 6 with value: 12.714415663613222.
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\2576784288.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\2576784288.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
C

In [14]:
xgb_params=xgb_study.best_params 
xgb_params['random_state'] = 42
xgb_params['metric'] = 'rmse'
xgb_params['n_estimators'] = 10000
xgb_params

{'lambda': 2.4827990514474725,
 'alpha': 0.31216401684148964,
 'colsample_bytree': 0.5,
 'subsample': 1.0,
 'learning_rate': 0.005,
 'max_depth': 7,
 'min_child_weight': 132,
 'random_state': 42,
 'metric': 'rmse',
 'n_estimators': 10000}

In [15]:
def cat_objective(trial,data=X,target=y):
    
    X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.30, random_state=42)
    param = {
        'loss_function': 'RMSE',
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
        'max_bin': trial.suggest_int('max_bin', 200, 400),
        #'rsm': trial.suggest_uniform('rsm', 0.3, 1.0),
        'subsample': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.006, 0.018),
        'n_estimators':  25000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9]),
        'random_state': 42,
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
    }
    model = CatBoostRegressor(**param) 
    
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=20,verbose=False)
    
    preds = model.predict(X_test)
    
    rmse = mean_squared_error(y_test, preds,squared=False)
    
    return rmse

In [16]:
cat_study = optuna.create_study(direction='minimize')
cat_study.optimize(cat_objective, n_trials=num_trials)

[I 2022-11-22 00:37:52,800] A new study created in memory with name: no-name-3693f7cf-b340-415a-83c8-d0f26d197fb7
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\742444196.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\742444196.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'subsample': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\742444196.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://gi

C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\742444196.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\742444196.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'subsample': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\742444196.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rat

C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\742444196.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\742444196.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'subsample': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\742444196.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rat

C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\742444196.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\742444196.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'subsample': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\742444196.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rat

In [17]:
cat_params=cat_study.best_params 
cat_params['random_state'] = 42
cat_params['loss_function'] = 'RMSE'
cat_params['n_estimators'] = 25000
cat_params.pop('bagging_fraction')
cat_params

{'l2_leaf_reg': 9.69076342640467,
 'max_bin': 224,
 'learning_rate': 0.017642646762641698,
 'max_depth': 9,
 'min_data_in_leaf': 18,
 'random_state': 42,
 'loss_function': 'RMSE',
 'n_estimators': 25000}

In [18]:
def gbr_objective(trial):
    X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.30, random_state=42)
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 1000, 10000, step=200),
        "learning_rate": trial.suggest_categorical('learning_rate', [0.01,0.03,0.05,0.1]),
        "max_depth": trial.suggest_int("max_depth", 3, 9, step=2),
        "random_state": 42,
    }
    model = GradientBoostingRegressor(**param)
    model.fit(X_train,y_train)
    
    preds = model.predict(X_test)
    
    rmse = mean_squared_error(y_test, preds,squared=False)
    
    return rmse

In [19]:
gbr_study = optuna.create_study(direction='minimize')
gbr_study.optimize(gbr_objective, n_trials=1)

[I 2022-11-22 00:54:30,942] A new study created in memory with name: no-name-690d5218-303a-4da3-a64b-adefefe12d94
[W 2022-11-22 01:06:24,668] Trial 0 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "C:\Users\bhara\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\bhara\AppData\Local\Temp\ipykernel_19880\3902487942.py", line 10, in gbr_objective
    model.fit(X_train,y_train)
  File "C:\Users\bhara\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 668, in fit
    n_stages = self._fit_stages(
  File "C:\Users\bhara\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 745, in _fit_stages
    raw_predictions = self._fit_stage(
  File "C:\Users\bhara\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 247, in _fit_stage
    tree.fit(X, residual, sample_weight=sample_weight, check_input=False)
  File "C:\Users\bhara\anaconda3\lib\site-pac

KeyboardInterrupt: 

In [ ]:
gbr_params=gbr_study.best_params 
gbr_params['random_state'] = 42
gbr_params

In [ ]:
def rf_objective(trial):
    X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.30, random_state=42)
    param = {
        "n_estimators": trial.suggest_int(name="n_estimators", low=100, high=1000),
        "max_depth": trial.suggest_int("max_depth", 3, 8),
        "max_features": trial.suggest_categorical(
            "max_features", choices=["auto", "sqrt", "log2"]
        ),
        "n_jobs": -1,
        "random_state": 42,
    }

    model = RandomForestRegressor(**param)
    
    model.fit(X_train,y_train)
    
    preds = model.predict(X_test)
    
    rmse = mean_squared_error(y_test, preds,squared=False)
    
    return rmse
    

In [ ]:
rf_study = optuna.create_study(direction='minimize')
rf_study.optimize(rf_objective, n_trials=num_trials)

In [ ]:
rf_params=rf_study.best_params 
rf_params['random_state'] = 42
rf_params['n_jobs'] = -1
rf_params

In [ ]:
def svr_objective(trial):
    X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.30, random_state=42)
    param = {
        'C': trial.suggest_loguniform('C', 0.3, 10.0),
        'epsilon': trial.suggest_float('epsilon', 0.05, 0.5),
    }
    
    model = SVR(**param)
    model.fit(X_train,y_train)
    
    preds = model.predict(X_test)
    
    rmse = mean_squared_error(y_test, preds,squared=False)
    
    return rmse
        

In [ ]:
svr_study = optuna.create_study(direction='minimize')
svr_study.optimize(svr_objective, n_trials=num_trials)

In [ ]:
svr_params=svr_study.best_params 
#svr_params['random_state'] = 42
#svr_params['n_jobs'] = -1
svr_params

In [ ]:
print('Hello World!')

In [21]:
xgb_model = XGBRegressor(**xgb_params)

In [22]:
xgb_model.fit(X,y)

[01:07:49] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "metric" } are not used.



XGBRegressor(alpha=0.31216401684148964, base_score=0.5, booster='gbtree',
             callbacks=None, colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.5, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=-1, grow_policy='depthwise', importance_type=None,
             interaction_constraints='', lambda=2.4827990514474725,
             learning_rate=0.005, max_bin=256, max_cat_threshold=64,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=7, max_leaves=0,
             metric='rmse', min_child_weight=132, missing=nan,
             monotone_constraints='()', n_estimators=10000, n_jobs=0, ...)

In [23]:
y_pred = xgb_model.predict(test_df)

In [24]:
y_pred = inv_boxcox(y_pred, y_train_lambda)

In [ ]:
this == that

In [26]:
sub_df = pd.read_csv('sample_submission.csv')
sub = pd.DataFrame({'id': sub_df.id, 'price_doc': y_pred})
sub.to_csv('sub004.csv', index = False)

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
level0 = list()
level0.append(('xgb', XGBRegressor(**xgb_params)))
level0.append(('lgbm', LGBMRegressor(**lgbm_params)))
level0.append(('svm', SVR(**svr_params)))
level0.append(('gbr', GradientBoostingRegressor(**gbr_params)))
level0.append(('rf', RandomForestRegressor(**rf_params)))
level0.append(('cat', CatBoostRegressor(**cat_params)))
level1 = LinearRegression()

In [ ]:
model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)

In [ ]:
model.fit(X, y)

In [ ]:
yhat = model.predict(test_df)